<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/AdaNet/experiments/wine_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install adanet

     |████████████████████████████████| 122kB 13.7MB/s 
     |████████████████████████████████| 163kB 21.8MB/s 
     |████████████████████████████████| 215kB 37.8MB/s 
  Created wheel for rednose: filename=rednose-1.3.0-cp36-none-any.whl size=12156 sha256=c46656b7488abdcbb8ddbcff18395652c569e63234923bc696ddbf5d91f10332
  Stored in directory: /root/.cache/pip/wheels/21/be/74/e74fbccdaf7ab983f847153b2708e4d1722801ce7265dfa643
  Created wheel for termstyle: filename=termstyle-0.1.11-cp36-none-any.whl size=4778 sha256=cea7b340c9252bc22e84a5261100c58e57276e459d242bae09bcb1270cd1473a
  Stored in directory: /root/.cache/pip/wheels/93/0c/cf/1d134a42237c338ee1d733c81b92d95f6d04d9531bf4c2a9a1
Successfully built rednose termstyle
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 4.5.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.9

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
%cd /gdrive/My Drive/機械学習練習/AutoML/Adanet/experiment
%ls

/gdrive/My Drive/機械学習練習/AutoML/Adanet/experiment
wine_cls.ipynb


# Wine分類モデルをAdaNetで構築

In [8]:
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import tensorflow as tf
import adanet

## データ読み込み

In [46]:
SEED = 2

wine_data = load_wine()

feature_names = wine_data.feature_names
target_names = wine_data.target_names

train_x, test_x, train_y, test_y = train_test_split(wine_data.data, wine_data.target,test_size=0.2, random_state=SEED)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=SEED)
print('train_x:{}, train_y:{}, val_x:{}, val_y:{}, test_x:{}, test_y:{}'.format(train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape))

train_x:(113, 13), train_y:(113,), val_x:(29, 13), val_y:(29,), test_x:(36, 13), test_y:(36,)


## AdaNet

In [15]:
# モデルの出力の定義
head= tf.estimator.MultiClassHead(n_classes=len(target_names),
                                  label_vocabulary=list(target_names))
head

In [19]:
estimator = adanet.AutoEnsembleEstimator(head=head,
                                         candidate_pool=lambda config:{
                                             'linear':
                                                tf.estimator.LinearEstimator(
                                                    head=head,
                                                    feature_columns=feature_names,
                                                    config=config
                                                ),
                                              'dnn':
                                                tf.estimator.DNNEstimator(
                                                    head=head,
                                                    feature_columns=feature_names,
                                                    config=config,
                                                    hidden_units=[20, 15, 10]
                                                )
                                         },
                                         max_iteration_steps=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
def input_evaluation_set(x, feature_names, y):
    '''
    Parameters
    -----
    x : np.ndarray(x, d)
    features_name : list : string
    y : np.ndarray(d)

    Returns
    -----
    features : dict{'feature_name1':np.ndarray(d), 'feature_name2':np.ndarray(d),,,,,,,}
    labels : np.ndarray(n)

    INFO
    -----
      ----usage example-----
      features = {'SepalLength': np.array([6.4, 5.0]),
                  'SepalWidth':  np.array([2.8, 2.3]),
                  'PetalLength': np.array([5.6, 3.3]),
                  'PetalWidth':  np.array([2.2, 1.0])}
      labels = np.array([2, 1])
      return features, labels
      -----
    '''
    features = {}
    for key, f in zip(feature_names, x.T):
      features[key] = f
    labels = y
    return features, labels

In [48]:
train_features, train_labels = input_evaluation_set(x=train_x, feature_names=feature_names, y=train_y)
val_features, val_labels = input_evaluation_set(x=val_x, feature_names=feature_names, y=val_y)
test_feature = input_evaluation_set(x=test_x, feature_names=feature_names, y=None)

In [49]:
def input_fn(features, labels, training=True, batch_size=256):
    '''
    トレーニング/検証データを提供する入力関数
    
    Parameters
    -----
    features : dict{'f_name':np.ndarray(d),,,,}
    labels : np.ndarray(d)

    Returns
    -----
    datasets : tf.data.Dataset
    '''
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
      dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [50]:
estimator.train(input_fn=lambda :input_fn(train_features, train_labels),
                steps=100)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9pd4gtz4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


ValueError: ignored